# Exercícios!

O código abaixo vai montar o Google Drive como um disco e torna-lo acessível para salvarmos os arquivos no drive! 

**É necessário apenas para quem usar o Google Colab**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


A classe abaixo nos ajudará a acompanhar o progresso do download.

##**Exercício 1**

1.   Com base no dataset disponibilizado para download, carrege 10 mil linhas de forma randômica.<br/>
    - https://dados-ml-pln.s3-sa-east-1.amazonaws.com/imdb-reviews-pt-br.csv
2.   Crie um classificador que consiga predizer o sentimeto associado a um texto novo (não pertencente a amostra)<br/>


In [0]:
import pandas as pd

df = pd.read_csv(
    'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/imdb-reviews-pt-br.csv'
).sample(10000, random_state=42)

df.shape

(10000, 4)

In [0]:
df.head()

,id,text_en,text_pt,sentiment
12532,12534,This was unusual: a modern-day film which was ...,Isso era incomum: um filme moderno que era ult...,pos
35445,35447,Some of my old friends suggested me to watch t...,Alguns dos meus velhos amigos sugeriram que eu...,neg
20279,20281,What a pleasure. This is really a parody. Only...,Que prazer. Isto é realmente uma paródia. Some...,pos
2969,2971,There are about ten minutes about half way thr...,"Há cerca de dez minutos a meio da Strangeland,...",neg
45161,45163,"Othello, the classic Shakespearen story of lov...","Otelo, a clássica história de Shakespearen sob...",pos


In [0]:
df.sentiment.value_counts()

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
vect.fit(df.text_pt)
text_vect = vect.transform(df.text_pt)

In [0]:
text_vect.shape

(10000, 66053)

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    text_vect, 
    df.sentiment,
    test_size = 0.2, 
    random_state = 13
)

12532    pos
35445    neg
20279    pos
2969     neg
45161    pos
        ... 
25219    neg
30693    neg
20915    pos
14933    pos
45388    pos
Name: sentiment, Length: 10000, dtype: object

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree = DecisionTreeClassifier(random_state=13)
tree.fit(X_train, y_train)

y_prediction = tree.predict(X_test)

acc = accuracy_score(y_prediction, y_test)

print(acc)

0.675


In [0]:
text = vect.transform([
    'Eu não consegui assistir até o fim. Esse filme é muito legal! '
])
tree.predict(text)[0]

'pos'

##**Exercício 2**

1.   Crie uma função que substitua uma ou mais palavras de um texto um uma palavra similar com base na sua classe gramatical.
2.   Extenda essa função para considerar uma substituição positiva ou negativa.

In [0]:
!pip install scipy==1.2.0
!pip install gensim==2.0.0
#Obs: pode ser necessário reiniciar o Runtime do colab

#!pip install spacy
#!python -m spacy download pt
!python -m spacy download pt_core_news_sm

In [0]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

class DLProgress(tqdm):
  last_block = 0

  def hook(self, block_num=1, block_size=1, total_size=None):
    self.total = total_size
    self.update((block_num - self.last_block) * block_size)
    self.last_block = block_num

In [0]:
url = 'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip'
#url = 'https://dados-ml-pln.s3-sa-east-1.amazonaws.com/cbow_s300.zip'

tar_gz_path = '/content/gdrive/My Drive/FIAP/NLP/cbow_s300.zip'

if not isfile(tar_gz_path):
  with DLProgress(unit='B', unit_scale=True, miniters=1, desc='Word2Vec Model') as pbar:
    urlretrieve(
        url,
        tar_gz_path,
        pbar.hook)

if not isfile('/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt'):     
  zip_ref = zipfile.ZipFile(tar_gz_path, 'r')
  zip_ref.extractall('/content/gdrive/My Drive/FIAP/NLP/')
  zip_ref.close()

In [0]:
!ls -all -h /content/gdrive/My\ Drive/FIAP/NLP/cbow_s300.txt

-rw------- 1 root root 2.5G May 16 00:32 '/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt'


In [0]:
# outro exemplo de baixar o arquivo e descompactar, só não esqueça de alterar o diretório apontando o aquivo para a raiz do colabo ao carregar o modelo
'''
!wget 'http://143.107.183.175:22980/download.php?file=embeddings/word2vec/cbow_s300.zip'
!ls -la # veja o nome do arquivo compactado salvo no download
!unzip 'download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip' # subistitua com nome do arquivo
!ls -la
'''

total 3501352
drwxr-xr-x 1 root root       4096 Jun  6 15:08  .
drwxr-xr-x 1 root root       4096 Jun  6 12:18  ..
-rw-r--r-- 1 root root 2656045531 Oct  4  2018  cbow_s300.txt
drwxr-xr-x 1 root root       4096 Jun  2 16:14  .config
-rw-r--r-- 1 root root  929305948 Jun  6 15:02 'download.php?file=embeddings%2Fword2vec%2Fcbow_s300.zip'
drwx------ 4 root root       4096 Jun  6 14:27  gdrive
drwxr-xr-x 1 root root       4096 May 29 18:19  sample_data


In [0]:
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format('/content/gdrive/My Drive/FIAP/NLP/cbow_s300.txt')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import spacy

pt = spacy.load('pt_core_news_sm')
#pt = spacy.load('pt')

def trocar_por_similar(frase, pos, sentido):
  doc = pt(frase)
  
  nova_frase = []
  for token in doc:
    if token.pos_ == pos:
      similares = model.most_similar_cosmul(positive=[token.text], negative=[sentido] )
      for similar, _ in similares:
        if similar != token.text:
          break
      nova_frase.append(similar)
    else:
      nova_frase.append(token.text)
  return ' '.join(nova_frase)  

In [0]:
pt('Você é muito alto')[3].pos_

'ADJ'

In [0]:
print(trocar_por_similar('Você é muito baixo', 'ADJ', 'feio'))

Você é muito elevar
